# OPENSTREETMAP DATA PROJECT

## Map Area

### San Francisco, CA
https://mapzen.com/data/metro-extracts/metro/san-francisco_california/

https://www.openstreetmap.org/relation/111968

#### I used the San Francisco metro data from mapzen which includes some of the surrounding cities and towns, such as Redwood City, San Mateo, Oakland, etc. I currently live in Oakland and have lived in the bay area for the past 2-3 years and this was a way for me to not only get to know this place better, find useful and interesting things about the area but also help make the data from this area a little cleaner. 

## Data Sampling

#### The original San Francisco XML file was around 80mb while unzipped and 1.31 GB in size after extraction. This was too big so we made 2 smaller samples from the dataset, a small sized sample around 10Mb and intermediate test sample of around 135Mb, using systematic sampling(selecting every kth top level element). We used these smaller files for our data auditing process.

## Data Auditing

#### To audit data such as address, I use the code from audit.py and subsequently find problems associated with the data we are using. We also use several functions within audit.py to update these problematic data entries into formats we want them in.


### Problems Encountered in the Map

##### While auditing the data I found problems both manually and programmatically. The main problems that I identified were 
##### Abbreviated street names (“Street vs. ST, Avenue vs. AVE”)
##### Inconsistent postal codes (“CA-94066”, “94117-9991”, “M4E 2V5”)
##### Incorrect  or invalid postal codes (All postal codes in the area begin with '94' but quite a few on the map data had postal codes from outside the area, sometimes, from Central and South California and on rare occasion, even from Canada.)
##### Fields such as 'user' and 'uid' were missing from some entries 

## Data Cleaning

#### During the auditing stage, after we have identified some problems, I use some update functions in audit.py to print the updated street names and zipcodes. I then use clean.py and data.py files in this part of the project to clean problematical data and return the updated data entries. The shape_element function in data.py makes sure that the node and way XML elements are cleaned and shaped into several datastructures. I also used the shape_element function to take care of some entries that had fields such as 'user' or 'uid' missing. In absence of the user information, I could not be sure about the reliability of these data entries and so decided to skip these entirely. This was done for both the node and way elements. The code responsible for this part is shown below.

     for attribute in NODE_FIELDS:
        # if the key exists
        if element.attrib.get(attribute):
            node_attribs[attribute] = element.attrib[attribute]
        else:
            # an empty return statement will: a) exit the function, b) with a return value of `None`
            return
            
    Pape Ave => Pape Avenue
    E 28th Ave => East 28th Avenue
    Veterans Blvd => Veterans Boulevard
    Mendell St => Mendell Street
    Woodside Plz => Woodside Plaza
    Marshlands Rd => Marshlands Road


#### The function process_map writes the cleaned data into 5 .csv files. These will then be used to build the SQL database so that we can run queries on it and do some analysis. The csv files are:

#####    nodes.csv
#####    node_tags.csv
#####    ways.csv
#####    way_nodes.csv
#####    way_tags.csv


### Updating abbreviated street names

#### I iterated over each word in an address, correcting them to their respective mappings names. The function update_name accepts the street name as an argument and returns the new street name as a string.

    def update_name(name, mapping):
        words = name.split()
        for w in range(len(words)):
            if words[w] in mapping:
                if words[w-1].lower() not in ['suite', 'ste.', 'ste', 'avenue', 'ave'] or words[0] in mapping: 
                #For example, don't update 'Avenue E' to 'Avenue East'
                    words[w] = mapping[words[w]]
        name = " ".join(words)
        return name
        
        CA 94560 => 94560
        CA 94080 => 94080
        90214 => None
        M4E 2V5 => None
        94118-4504 => 94118

### Updating zipcodes

#### Like mentioned above Postal codes were inconsistent in their formatting and sometimes invalid for this area . Using the update_zipcode function, I updated the zipcodes to the 5 digit desirable format by trimming the leading and trailing characters of those 5 digits. The function also set a restriction on those 5 digits by making the first digits '94' such as to eliminate zipcodes not from the area.

    def update_zipcode(postcode):
        try:
            postcode = re.compile(r'94\d\d\d').search(postcode).group()
        except AttributeError:
            postcode = 'None'
        return postcode

## Creating Tables and Database

#### In this stage of the project, I used the 5 csv files we had earlier created and imported the data from there into the tables we created for the 'openstreetmap.db' database. I used pandas for this as the read_csv and to_sql functionality within pandas made this simple and easy. Snippets of the code I used for this is given below.

    sqlite_file = 'openstreetmap.db'
    db = sqlite3.connect(sqlite_file)
    c = db.cursor()
    
    c.execute('''CREATE TABLE nodes(id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT)''')
    db.commit()
    
    df_nodes = pd.read_csv('nodes1.csv',encoding = "utf-8")
    df_nodes.to_sql('nodes', db, if_exists='append', index=False)

## Data Exploration

#### Once my database was ready, I used SQL queries to explore the relatively cleaner San Francisco dataset.

#### I used the code below to run a query on the entire cleaned San Francisco database to find the counts of each postalcode by grouping.

    c.execute("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM node_tags UNION ALL \
    SELECT * FROM way_tags) tags WHERE tags.key='postcode' GROUP BY tags.value ORDER BY count DESC;")
    pprint.pprint(c.fetchall())

    [(u'94122', 5120),
     (u'94611', 2989),
     (u'94116', 2405),
     (u'94117', 1459),
     (u'94610', 1355),

#### The region with the most entries is the Outer Sunset, Inner Sunset district near Golden Gate Park. Although the ouput above only shows the top 5, it is nice to report that no zipcode that shouldn't have been there ended up showing in our database query. There were only 87 entries with no postal code or 'None' and this is because these originally had invalid, incorrect or no zipcodes provided.

#### I also ran a query to check the count of entries grouped by city usign the following code

    c.execute("SELECT tags.value,COUNT(*) as numcount FROM (SELECT * FROM node_tags UNION ALL SELECT * FROM way_tags) as \   tags WHERE tags.key='city' GROUP BY tags.value ORDER BY numcount DESC;")
    pprint.pprint(c.fetchall())

    [(u'Redwood City', 23479),
     (u'San Francisco', 19084),
     (u'Berkeley', 5756),
     (u'Piedmont', 3811),
     (u'Palo Alto', 1642),

#### It was interesting to see that the city with most entries was Redwood city and not San Francisco like I expected. The output above lists only the top 5 from the whole output. I noticed that some city names showed up twice and that is because of inconsistent usage of upper and lower case letters. 

## Data Overview and Additional Ideas

#### Below is a list of the files and sizes of the datasets we have been working with. In addition, there are SQL queries that we performed on the San Francisco database we created.

### File Sizes

In [ ]:
sfo.osm---------------->1.31GB
sample-sfo.osm------>10.4MB
test-sfo.osm---------->136MB
openstreetmap.db-->734MB
nodes1.csv----------->537MB
node_tags1.csv----->9.5MB
ways1.csv------------>49MB
way_tags1.csv------>59MB
way_nodes1.csv---->188MB

### Number of Nodes

    c.execute("SELECT COUNT(*) FROM nodes;")
    pprint.pprint(c.fetchall())
    
    [(6636823,)]

### Number of Ways

    c.execute("SELECT COUNT(*) FROM ways;")
    pprint.pprint(c.fetchall())
    
    [(827195,)]

### Number of Unique Users

    c.execute("SELECT COUNT(DISTINCT(allusers.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) \
    as allusers;")
    pprint.pprint(c.fetchall())
    
    [(2853,)]

### Most Prolific Users-Top 10

    c.execute("SELECT allusers.user,COUNT(*) as num FROM (SELECT user,uid FROM nodes UNION ALL \
    SELECT user,uid FROM ways) as allusers GROUP BY allusers.uid ORDER BY num DESC LIMIT 10;")      
    pprint.pprint(c.fetchall())
    
    [(u'andygol', 1495891),
     (u'ediyes', 887577),
     (u'Luis36995', 679544),
     (u'dannykath', 546167),
     (u'RichRico', 415780),

### List of Amenities in the SF metro by count

    c.execute("SELECT value,count(*) as numcount FROM node_tags WHERE key='amenity' GROUP BY value ORDER BY numcount \
    DESC LIMIT 50;")
    pprint.pprint(c.fetchall())
    
    [(u'restaurant', 2993),
     (u'bench', 1231),
     (u'cafe', 1020),
     (u'place_of_worship', 695),
     (u'post_box', 691),

### Locations of EV charging stations

#### Something that I found interesting was the charging stations. In the SF metro area there are lot of electric vehicles(Tesla, Prius) and because charging stations aren't as frequent as gas stations, it becomes important for the owner and the car to know where possible stations are. Tesla's in built computer system can find charging stations within its range so I decided to use this data to find the gps coordinates of the 23 charging stations in this map area. The query I used to find that is below.

    c.execute("SELECT node_tags.value,node_tags.id,nodes.lat,nodes.lon FROM nodes JOIN node_tags \
    ON nodes.id=node_tags.id WHERE node_tags.value='charging_station';") 
    pprint.pprint(c.fetchall())
    
    [(u'charging_station', 3096916602L, 37.7153342, -122.21446979999999),
     (u'charging_station', 3096916603L, 37.7153474, -122.21445159999999),
     (u'charging_station', 3143189345L, 37.53352039999999, -122.27272090000001),
     (u'charging_station', 3149810839L, 37.7119026, -122.2108358),
     (u'charging_station', 3149810840L, 37.7119155, -122.21085959999999),

### 20 Most Popular Restaurant Cuisines

#### The San Francisco Metro area is big on eating out and has a very vibrant and diverse food scene. Keeping that in mind, I ran a query to find the 20 most popular restaurant cuisines by grouping and counting how many restaurants are classified within each group. 

    c.execute("SELECT node_tags.value, COUNT(*) as num FROM node_tags JOIN (SELECT DISTINCT(id) FROM node_tags \
    WHERE value = 'restaurant') AS distinctones ON node_tags.id = distinctones.id WHERE node_tags.key = 'cuisine' \
    GROUP BY node_tags.value ORDER BY num DESC LIMIT 20;")
    pprint.pprint(c.fetchall())
    
    [(u'mexican', 200),
     (u'chinese', 166),
     (u'pizza', 159),
     (u'japanese', 138),
     (u'italian', 128),

### Places to stay

#### The SF metro area is a big tourist destination and as such, I was interested to find the GPS location of most places one could stay here.  

    c.execute("SELECT node_tags.value,nodes.lat,nodes.lon FROM nodes JOIN node_tags ON nodes.id=node_tags.id \
    WHERE node_tags.key='tourism' and node_tags.value='camp_site' or node_tags.value='hotel' or node_tags.value='hostel' \
    or node_tags.value='motel' or node_tags.value='inn' ORDER BY node_tags.value;") 
    pprint.pprint(c.fetchall())
    
    [(u'camp_site', 37.883732200000004, -122.2304357),
     (u'hostel', 37.536639, -122.518833),
     (u'hotel', 37.4990743, -122.24090770000001),
     (u'inn', 37.779619, -122.5092453),
     (u'motel', 37.5760917, -122.31890179999999),

##### NOTE: Most outputs are snippets of the original.

## Conclusion

#### After going through the different stages of the data wrangling process, I cleaned the data well enough for the purposes of the project but this dataset is not 100% clean. For example, we noticed that city names may have letters in different cases. There is most likely similar inconsistencies in the state name as well. Another challenge that I dealt with during this project was missing user/uid fields from data entries which therefore, might not be reliable and so I eliminated those entries.
#### The OpenStreetMap project is massive and kudos must be given to its millions of users who have collected and compiled thousands of gigabytes of data by adding so many different layers of detail in the map data. I really liked the GPS latitude/longitude information available and decided to use it in my exploration. I must raise a hint of caution here though. If the GPS data is anything like the other fields, chances are that lot of these could be incorrect, inaccurate or invalid.

#### Over the course of this project, I used data wrangling techniques, such as data validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data. Then I wrote the cleaned XML dataset into CSV files, created a database with tables and imported the csv files into those tables. Once the database is ready, I conducted SQL queries to provide an overview of the dataset and do some interesting exploration. Using that insight I feel it would be great if the OSM project could add a few formal measures of data quality and assurance while accepting data from its users, for example, conforming to a set schema, maintaining consistency and uniformity among values such as zipcode, registering only entries that are complete and have no missing fields, and incorporating cross-field constraints to ensure data like city name, state name and zipcode is more accurate.
#### The above mentioned recommendations will create challenges themselves. Firstly, for all of these steps to be executed, OSM probably has to hire and pay a bunch of people to develop and design a platform based on those requirements, secondly, the designing and the restriction will probably slow down the rate at which new map data is added onto the project, thirdly, some users might not like the idea and OSM might lose some of its user base and finally, this will be a slight divertion from their core belief of making the map available to everyone to edit,fix,update,use or download. 


## References

##### https://gist.github.com/carlward/54ec1c91b62a5f911c42#problems-encountered-in-the-map
##### https://discussions.udacity.com/t/p3-project-combining-auditing-cleaning-and-csv-creation/231037/2
##### https://discussions.udacity.com/t/osm-data-project-getting-started-running-locally/232476/11
##### https://github.com/davidventuri/udacity-dand/blob/master/p3/report.md
##### https://mapzen.com/data/metro-extracts/metro/san-francisco_california/
##### https://www.openstreetmap.org/relation/111968